In [0]:
### Autoloader is built on top of spark streaming
## this will transfer the data and creates a checkpoint which takes care of idempotent behaviour
## It will scan the data and write the schema in schema location
## When schema is changed then it will add the columns at the end of file after scanning.
## Every time it puts schema to data location it caches it and it will not update the cache.
## Schema loader will take care of newly added columns

In [0]:
## Recommended to put schema and checkpoint location in singles folder as easy to maintain and easy to manage

## INCREMENTAL DATA INGESTION

In [0]:
# %sql
# CREATE VOLUME IF NOT EXISTS project.raw.gold

In [0]:
dbutils.widgets.text("src", "")

In [0]:
src_value = dbutils.widgets.get("src")
src_value

In [0]:
df = spark.readStream.format("cloudFiles")  \
          .option("cloudFiles.format","csv")  \
          .option("cloudFiles.schemaLocation", f"/Volumes/project/bronze/bronzevolume/{src_value}/checkpoint")  \
          .option("cloudFiles.schemaEvolutionMode", "rescue")   \
          .load(f"/Volumes/project/raw/raw/rawsrc/{src_value}/")

In [0]:
df.writeStream.format("delta")  \
    .outputMode("append") \
    .trigger(once=True) \
    .option("checkpointLocation",f"/Volumes/project/bronze/bronzevolume/{src_value}/checkpoint") \
    .option("path", f"/Volumes/project/bronze/bronzevolume/{src_value}/data") \
    .start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/project/bronze/bronzevolume/flights/data/`